In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import os.path
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
parent_url = 'https://www.prokabaddi.com'
teams_rel_loc= 'teams'
players_rel_loc = 'players' 
teams = ['bwarriors','bbulls']
team_loc_dict = {"bwarriors":"bengal-warriors-profile-4",
                "bbulls" : "bengaluru-bulls-profile-1",
              "ddelhi": "dabang-delhi-kc-profile-2"}
base_op_dir = './data/'

In [ ]:
class utils:
    def make_url(base_url, *rel_locs):
        final_url = base_url
        for loc in rel_locs:
            final_url = final_url + '/' + loc
        return final_url

In [ ]:
def write_to_file(filenameprefix, name, list_of_statvals, header):
    print(header)
    filename  = filenameprefix+'.csv'
    filepath = '/'.join([base_op_dir,filename])
    list_of_statvals.append(name)
    file_exists = os.path.isfile(filepath)
    with open(filepath, 'w') as season_file:
            wr = csv.writer(season_file, quoting=csv.QUOTE_ALL)
            if not file_exists:
                wr.writerow(header)
            wr.writerow(list_of_statvals)

In [ ]:
def init_stats(browser):
    browser.get(player_url)
    time.sleep(5)
    stats_page = browser.find_element_by_xpath('//div[@data-tabtype="stats"]')
    print(stats_page)
    stats_page.click()

In [ ]:
def get_stats_for_player(browser):
    right_header_section = soup.find('div', class_='si-right').find("div", class_='si-tbl-header')
    col_name_divs = right_header_section.findAll('div',class_='si-data-block')
    col_names = []
    row_names = []
    for col in col_name_divs:
        col_names.append(col.text)
    col_names = [name.replace(' ','') for name in col_names]
    
    prev_section = right_header_section
    stats_dict = dict()
    while 4<5:
        right_data_section = prev_section.findNext("div", class_='si-tbl-wraper')
        if right_data_section:
            all_values = right_data_section.findAll('div', class_='si-tbl-data')
            i=0
            for col_val in all_values:
                val_list=[]
                all_vals_for_column = col_val.findAll('div', class_='si-data-block')
                for val in all_vals_for_column:
                    val_list.append(val.get_text().strip())
                if col_names[i] not in stats_dict:
                    stats_dict['%s' %(col_names[i])]=val_list
                else:
                    stats_dict['%s' %(col_names[i])].extend(val_list)
                i=i+1
        else:
            break
        prev_section = right_data_section
        
    row_name_divs = soup.find('div', class_='si-left').findAll("div", class_='si-tbl-wraper')
    row_names=[]
    for name_div in row_name_divs:
        temp = name_div.findAll("div", class_='si-data-block')
        for row_div in temp:
            row_names.append(row_div.find(text=True, recursive=False))
    row_names.pop(0)
    row_names.append('player_name')
    
    return(col_names, row_names, stats_dict)

In [ ]:
team_url = 'https://www.prokabaddi.com/teams/bengaluru-bulls-profile-1/players' #Convert to a loop, of all teams
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get(team_url)
time.sleep(3)
soup = BeautifulSoup(browser.page_source, "html.parser")
name_list = soup.find('div', 'si-page-plyrWrp').findAll("a", {"class": "si-plyrBox"})
all_player_urls=[]
for name_tag in name_list:
    all_player_urls.append(utils.make_url(parent_url,name_tag['href']))

In [ ]:
for player_url in all_player_urls:
    print(player_url)
    init_stats(browser)
    soup = BeautifulSoup(browser.page_source, "html.parser")
    player_name = soup.find('span', class_='si-name').get_text()
    col_names,row_names,data = get_stats_for_player(browser)
    for season_val in col_names:
        write_to_file(season_val, player_name, data[season_val], row_names)
browser.close()

In [ ]:
browser.close()